In [90]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

In [91]:
df = pd.read_csv('BitcoinHeistData.csv')

In [92]:
df.head()

,address,year,day,length,weight,count,looped,neighbors,income,label
0,111K8kZAEnJg245r2cM6y9zgJGHZtJPy6,2017,11,18,0.008333,1,0,2,100050000.0,princetonCerber
1,1123pJv8jzeFQaCV4w644pzQJzVWay2zcA,2016,132,44,0.000244,1,0,1,100000000.0,princetonLocky
2,112536im7hy6wtKbpH1qYDWtTyMRAcA2p7,2016,246,0,1.000000,1,0,2,200000000.0,princetonCerber
3,1126eDRw2wqSkWosjTCre8cjjQW8sSeWH7,2016,322,72,0.003906,1,0,2,71200000.0,princetonCerber
4,1129TSjKtx65E35GiUo4AYVeyo48twbrGX,2016,238,144,0.072848,456,0,1,200000000.0,princetonLocky


In [93]:
df['address'].value_counts()

1LXrSb67EaH1LGc6d6kWHq8rgv4ZBQAcpU    420
16cVG72goMe4sNqZhnpmnqfCMZ1uSFbUit    261
12wQZTDmA8onM3sEt4jwcvzDxnNXxD8Vza    207
12YursV58dRT2c9iuZg3jEWfwgTDamBcnd    183
1LEq4WmpCrqBd7V3PywE2nvFUFC3QTe52x    176
                                     ... 
14m4NjEQjLKrcjtN3doN7TgNZi3nbvPnkL      1
1CJrNRSNJepexvLFt3wSKZkzrHRag2UMCA      1
1Fsi7R5115vXKcSmFEoDUqqmEW4oT2W5AV      1
1GTkpRYXAK71c5DP2V7irDmYtvmhS46h29      1
3LFFBxp15h9KSFtaw55np8eP5fv6kdK17e      1
Name: address, Length: 2631095, dtype: int64

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2916697 entries, 0 to 2916696
Data columns (total 10 columns):
 #   Column     Dtype  
---  ------     -----  
 0   address    object 
 1   year       int64  
 2   day        int64  
 3   length     int64  
 4   weight     float64
 5   count      int64  
 6   looped     int64  
 7   neighbors  int64  
 8   income     float64
 9   label      object 
dtypes: float64(2), int64(6), object(2)
memory usage: 222.5+ MB


No null values present in the Datset

In [95]:
df.isnull().sum()

address      0
year         0
day          0
length       0
weight       0
count        0
looped       0
neighbors    0
income       0
label        0
dtype: int64

Randomly shuffle the Dataset

In [ ]:
df = df.sample(frac=1)

Encoding 'address' column into numerical column

In [141]:
encoder = LabelEncoder()
df['address'] = encoder.fit_transform(df['address'])

In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2916697 entries, 11381 to 349553
Data columns (total 10 columns):
 #   Column     Dtype  
---  ------     -----  
 0   address    int32  
 1   year       int64  
 2   day        int64  
 3   length     int64  
 4   weight     float64
 5   count      int64  
 6   looped     int64  
 7   neighbors  int64  
 8   income     float64
 9   label      object 
dtypes: float64(2), int32(1), int64(6), object(1)
memory usage: 233.7+ MB


In [143]:
df.head()

,address,year,day,length,weight,count,looped,neighbors,income,label
11381,689922,2016,189,62,5.651403e-07,2,0,1,2.500000e+08,princetonLocky
267948,620511,2011,237,6,1.337209e-01,46,0,2,1.003000e+09,white
2670125,1079102,2018,84,0,1.052632e-02,1,0,2,9.298099e+07,white
2581349,2534943,2017,360,144,8.499852e-03,5647,0,2,3.023130e+07,white
42631,2345303,2011,3,0,5.000000e-01,1,0,2,2.000000e+09,white


In [144]:
df['label'].value_counts()

white                          2875284
paduaCryptoWall                  12390
montrealCryptoLocker              9315
princetonCerber                   9223
princetonLocky                    6625
montrealCryptXXX                  2419
montrealNoobCrypt                  483
montrealDMALockerv3                354
montrealDMALocker                  251
montrealSamSam                      62
montrealCryptoTorLocker2015         55
montrealGlobeImposter               55
montrealGlobev3                     34
montrealGlobe                       32
montrealWannaCry                    28
montrealRazy                        13
montrealAPT                         11
paduaKeRanger                       10
montrealFlyper                       9
montrealXTPLocker                    8
montrealVenusLocker                  7
montrealXLockerv5.0                  7
montrealCryptConsole                 7
montrealEDA2                         6
montrealJigSaw                       4
paduaJigsaw              

# Q3 Part-1

In [204]:
def train_test_validation_split(dataset: pd.DataFrame, size):
    n = dataset.shape[0]
    train_size = round((size[0]/100) * n)
    valid_size = round((size[1]/100) * n)
    test_size = round((size[2]/100) * n)
    
    x_train = dataset.iloc[0:train_size].drop('label', axis=1)
    y_train = dataset.iloc[0:train_size]['label']
    x_valid = dataset.iloc[train_size: train_size + valid_size].drop('label', axis=1)
    y_valid = dataset.iloc[train_size: train_size + valid_size]['label']
    x_test = dataset.iloc[train_size + valid_size:].drop('label', axis=1)
    y_test = dataset.iloc[train_size + valid_size:]['label']
    return x_train, y_train, x_valid, y_valid, x_test, y_test
    

In [205]:
x_train, y_train, x_valid, y_valid, x_test, y_test = train_test_validation_split(df, [70,15,15])

In [206]:
def DecisionTreeAlgorithm(split_criteria, x_train, y_train, x_valid, y_valid, x_test, y_test):
    print(f"Training & Testing the Decision Tree with '{split_criteria}' criteria...")
    print("Accuracy of Decision Tree classifers with various depth is shown below.\n")
    
    for depth in  [4, 8, 10, 15, 20]:
        tree = DecisionTreeClassifier(max_depth=depth, criterion=split_criteria)
        tree.fit(x_train, y_train)
        y_pred = tree.predict(x_test)
        
        print(f"Accuracy with Depth = {depth} is: {(y_pred == y_test).sum() / y_test.shape[0]}")

### Training Decision trees using the Gini index (for various depths)

In [170]:
DecisionTreeAlgorithm(split_criteria='gini', x_train=x_train, y_train=y_train,
                    x_valid=x_valid, y_valid=y_valid, x_test=x_test, y_test=y_test)

Training & Testing the Decision Tree with 'gini' criteria...
Accuracy of Decision Tree classifers with various depth is shown below.

Accuracy with Depth = 4 is: 0.9860389847864248
Accuracy with Depth = 8 is: 0.9865052662375658
Accuracy with Depth = 10 is: 0.9872938304564073
Accuracy with Depth = 15 is: 0.9885143907255705
Accuracy with Depth = 20 is: 0.9881852508777063


### Training Decision trees using the Entropy (for various depths)

In [169]:
DecisionTreeAlgorithm(split_criteria='entropy', x_train=x_train, y_train=y_train,
                    x_valid=x_valid, y_valid=y_valid, x_test=x_test, y_test=y_test)

Training & Testing the Decision Tree with 'entropy' criteria...
Accuracy of Decision Tree classifers with various depth is shown below.

Accuracy with Depth = 4 is: 0.9860024136922176
Accuracy with Depth = 8 is: 0.9863932672615565
Accuracy with Depth = 10 is: 0.98760239906378
Accuracy with Depth = 15 is: 0.9890766712990052
Accuracy with Depth = 20 is: 0.9881075373025161


Greatest accuracy is observed for Decision Tree with "Depth" as 15 with criteria as "Entropy".


# Q3 Part-2

In [212]:
num_trees = 100

In [232]:
def train_trees(x_train, y_train, num_trees):
    trees = [DecisionTreeClassifier(max_depth=3, criterion='entropy') for i in range(num_trees)]
    
    # random indices for bootstrap samples
    m = x_train.shape[0]
    indices = np.arange(m, dtype=np.int16)    
    
    for i in range(num_trees):
        # Selecting n random samples with replacement from training set
        random_indices = np.random.choice(indices, m//2)
        # Bootstrap training data
        x_bootstrap = x_train.iloc[random_indices]
        y_bootstrap = y_train.iloc[random_indices]
                
        # Train/fit the Data on the Trees
        trees[i].fit(x_bootstrap, y_bootstrap)
    
    return trees

In [227]:
def predict_outputs(trees, x_test, y_test, num_trees):
    trees_predictions = np.empty(shape=(num_trees, x_test.shape[0]), dtype='object')
    for i in range(num_trees):
        trees_predictions[i] = trees[i].predict(x_test)
    
    y_prediction = np.empty(shape=(x_test.shape[0]), dtype='object')
    for i in range(x_test.shape[0]):
        y_prediction[i] = pd.Series(trees_predictions[:,i]).value_counts().index[0]
    
    return y_prediction, trees_predictions

In [233]:
trees = train_trees(x_train=x_train, y_train=y_train, num_trees=num_trees)

In [234]:
y_prediction, trees_prediction = predict_outputs(
                trees=trees, x_test=x_test, y_test=y_test, num_trees=num_trees)

In [235]:
accuracy = (y_prediction == y_test).sum() / y_prediction.shape[0]

print(f"Random Forest with '{num_trees} Decision-Trees each of depth 3'.")
print(f'Accuracy of Random Forest is: {accuracy}')

Random Forest with '100 Decision-Trees each of depth 3'.
Accuracy of Random Forest is: 0.9860024136922176
